In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv("/Users/shrish/Desktop/ATC - Visibility Prediction/Dataset/NOAA-JKF-AP.csv")

In [4]:
df.head(2)

,DATE,VISIBILITY,DRYBULBTEMPF,WETBULBTEMPF,DewPointTempF,RelativeHumidity,WindSpeed,WindDirection,StationPressure,SeaLevelPressure,Precip
0,2010-09-18 20:51:00,10.0,64,59,55,73,6,160,30.17,30.19,0.0
1,2010-09-18 21:51:00,10.0,63,58,55,75,3,160,30.17,30.19,0.0


In [5]:
df = df.drop(['DATE','WETBULBTEMPF','DewPointTempF',"StationPressure"],axis=1)

In [6]:
df.tail(2)

,VISIBILITY,DRYBULBTEMPF,RelativeHumidity,WindSpeed,WindDirection,SeaLevelPressure,Precip
6255,10.0,80,41,3,50,30.0,0.0
6256,10.0,78,47,9,190,30.0,0.0


In [7]:
X =df.drop('VISIBILITY', axis=1)
y =df['VISIBILITY']

In [8]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
X_train_scaled

array([[ 2.0922252 , -1.69721753,  1.29648858,  0.71892011, -0.4214487 ,
        -0.19703531],
       [ 0.71442105, -1.44252906,  1.29648858,  1.08178771, -0.54052015,
        -0.19703531],
       [-1.21450477, -0.0672113 , -0.51239715,  0.80963701,  0.5311229 ,
        -0.19703531],
       ...,
       [ 1.1966525 ,  0.64591643,  0.54278619, -0.2789658 , -0.10392483,
        -0.19703531],
       [ 0.98998188, -0.9840898 ,  0.24130524, -0.5511165 ,  1.2455516 ,
        -0.19703531],
       [ 1.05887208,  0.64591643, -0.96461859, -0.9139841 ,  0.25328952,
        -0.19703531]])

In [13]:
n_clusters = 3  # You may need to adjust this
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
train_clusters = kmeans.fit_predict(X_train_scaled)
test_clusters = kmeans.predict(X_test_scaled)

In [14]:
X_train_with_clusters = np.column_stack((X_train, train_clusters))
X_test_with_clusters = np.column_stack((X_test, test_clusters))

In [15]:
X_train_with_clusters

array([[77.  , 29.  , 21.  , ..., 29.88,  0.  ,  0.  ],
       [57.  , 34.  , 21.  , ..., 29.85,  0.  ,  0.  ],
       [29.  , 61.  ,  9.  , ..., 30.12,  0.  ,  0.  ],
       ...,
       [64.  , 75.  , 16.  , ..., 29.96,  0.  ,  2.  ],
       [61.  , 43.  , 14.  , ..., 30.3 ,  0.  ,  1.  ],
       [62.  , 75.  ,  6.  , ..., 30.05,  0.  ,  1.  ]])

In [16]:
# Train a Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_with_clusters, y_train)

RandomForestRegressor(random_state=42)

In [17]:
# Make predictions
y_pred = rf_model.predict(X_test_with_clusters)

In [18]:
r2 = r2_score(y_test, y_pred)
print(f"R2 score: {r2}")

R2 score: 0.7190864923521607


In [19]:
# Optional: Train separate models for each cluster
cluster_models = {}
for cluster in range(n_clusters):
    cluster_mask = train_clusters == cluster
    cluster_X = X_train_scaled[cluster_mask]
    cluster_y = y_train[cluster_mask]
    
    cluster_model = RandomForestRegressor(n_estimators=100, random_state=42)
    cluster_model.fit(cluster_X, cluster_y)
    cluster_models[cluster] = cluster_model

In [20]:
# Predict using cluster-specific models
y_pred_cluster = np.zeros_like(y_test)
for cluster in range(n_clusters):
    cluster_mask = test_clusters == cluster
    cluster_X = X_test_scaled[cluster_mask]
    y_pred_cluster[cluster_mask] = cluster_models[cluster].predict(cluster_X)

In [21]:
# Calculate R2 score for cluster-specific approach
r2_cluster = r2_score(y_test, y_pred_cluster)
print(f"R2 score (cluster-specific models): {r2_cluster}")

R2 score (cluster-specific models): 0.7308467270427363


In [22]:
from sklearn.tree import DecisionTreeRegressor

In [23]:
dt=DecisionTreeRegressor(max_depth=5, min_samples_leaf=12, min_samples_split=25)

In [24]:
rf_model.fit(X_train_with_clusters, y_train)

RandomForestRegressor(random_state=42)

In [25]:
r2 = r2_score(y_test, y_pred)
print(f"R2 score: {r2}")

R2 score: 0.7190864923521607
